# Práctica 5.2 Transformación de datos  categóricos

## Dataset 2: Balance Scale Data Set
Recuperado de: http://archive.ics.uci.edu/ml/machine-learning-databases/nursery/

## Recipe
Los modelos de aprendizaje automático sólo pueden manejar variables numéricas. Por lo tanto, si una columna tiene una variable categórica, los modelos sólo funcionarán cuando esté codificada como variable numérica. La forma de codificar es uno de los elementos clave del desempeño en la modelación a partir de los datos.

## Transformación de variables numéricas
Es convertir una variable categórica en una variable numérica. La transformación de variables categóricas es casi obligatoria para la mayoría de los modelos de aprendizaje de máquinas porque sólo pueden manejar valores numéricos. También se denomina codificación (encoding), o en la minería de textos, la incrustación (embedding). (Tarea encoding y embedding)

## Información general sobre la transformación de datos
Existen muchas técnicas de transformación de datos, entre ellas, hay muchas que utilizan parámetros, como la media y la desviación estándar de la normalización o una tabla de conversión en la codificación de categorías. Un error común en el uso de la transformación variable es transformar el conjunto de entrenamiento y el conjunto de pruebas por separado utilizando parámetros diferentes. La transformación de los datos de entrenamiento y los datos de prueba debe hacerse utilizando los mismos parámetros, y normalmente se obtienen del conjunto de entrenamiento; de lo contrario, no podemos comparar los resultados justamente.

![alt text](0.png "Title")


Algunos dirán más estrictamente que cuando se trabaje con validación cruzada, los parámetros de transformación se derivarán sólo del volúmen de entrenamiento (k-fold), entonces los datos del volúmen de validación se transformarán por esos parámetros, en lugar de transformar todos los datos de entrenamiento antes de la validación cruzada. Ese enfoque podría ser necesario cuando se prevé que hay enormes diferencias en la distribución de los datos entre los volúmen. La presencia de valores atípicos es una posible razón para dar parámetros diferentes, en particular en las técnicas de transformación sensibles a los valores atípicos, como la escala mín-máx.

De cualquier manera, los científicos de datos siempre tienen que preguntarse cuál es el enfoque más razonable para los datos a los que se enfrentan.


## Codificación de un paso (one-hot encoding)

La codificación de un paso es un enfoque para convertir una columna categórica en múltiples columnas binarias (0 o 1), tantas como el número de niveles o categorías distintas de la columna original. Por ejemplo, si hay cuatro niveles en la variable categórica, la codificación de un paso creará cuatro nuevas columnas, cada una de las cuales tiene 0 o 1 y representa si la columna original tiene el nivel.

![alt text](1.png "Title")


Una desventaja de la codificación de un paso es que el número de columnas se incrementa fácilmente con muchos niveles distintos. Una posible solución es agrupar algunos niveles sobre la base del conocimiento del dominio o agrupar los niveles poco frecuentes en "otro" nivel.


In [1]:
import pandas as pd
df = pd.read_csv('car.data', names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class value'])
df.head()

,buying,maint,doors,persons,lug_boot,safety,class value
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


La codificación de un paso se puede hacer la función get_dummies de pandas o con el OneHotEncoder de scikit-learn.

In [2]:
pd.get_dummies(df,columns=['buying'])

,maint,doors,persons,lug_boot,safety,class value,buying_high,buying_low,buying_med,buying_vhigh
0,vhigh,2,2,small,low,unacc,0,0,0,1
1,vhigh,2,2,small,med,unacc,0,0,0,1
2,vhigh,2,2,small,high,unacc,0,0,0,1
3,vhigh,2,2,med,low,unacc,0,0,0,1
4,vhigh,2,2,med,med,unacc,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1723,low,5more,more,med,med,good,0,1,0,0
1724,low,5more,more,med,high,vgood,0,1,0,0
1725,low,5more,more,big,low,unacc,0,1,0,0
1726,low,5more,more,big,med,good,0,1,0,0


### What is One Hot Encoding?

A one hot encoding is a representation of categorical variables as binary vectors. This first requires that the categorical values be mapped to integer values. Then, each integer value is represented as a binary vector that is all zero values except the index of the integer, which is marked with a 1.

In [3]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

values = df['maint']
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
df_1 = pd.DataFrame(onehot_encoded, columns=['maint_vhigh', 'maint_high', 'maint_med', 'maint_low'])
df_1

,maint_vhigh,maint_high,maint_med,maint_low
0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
...,...,...,...,...
1723,0.0,1.0,0.0,0.0
1724,0.0,1.0,0.0,0.0
1725,0.0,1.0,0.0,0.0
1726,0.0,1.0,0.0,0.0


## Codificación de etiquetas (label encoding)

La codificación de etiquetas es un enfoque para convertir los niveles en enteros. Este enfoque no es apropiado en la mayoría de los algoritmos de aprendizaje automático porque la cantidad de valor transformado en realidad no tiene nada que ver con la variable objetivo, excepto los modelos basados en el árbol de decisión que pueden ser capaces de dividir la columna numérica transformada varias veces con la estratificación del nodo del árbol. Además, en el caso de que la variable categórica tenga una naturaleza "ordinal", por ejemplo, Frío < Calor < Caliente < Muy caliente, la codificación de etiquetas puede funcionar potencialmente mejor que otras técnicas de codificación.

![alt text](2.png "Title")

Se puede implementar la codificación de etiquetas con la función LabelEncoder de scikit-learn.

In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['lug_boot'])
df['lug_boot_Label_Encoder'] = le.transform(df['lug_boot'])
df

,buying,maint,doors,persons,lug_boot,safety,class value,lug_boot_Label_Encoder
0,vhigh,vhigh,2,2,small,low,unacc,2
1,vhigh,vhigh,2,2,small,med,unacc,2
2,vhigh,vhigh,2,2,small,high,unacc,2
3,vhigh,vhigh,2,2,med,low,unacc,1
4,vhigh,vhigh,2,2,med,med,unacc,1
...,...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good,1
1724,low,low,5more,more,med,high,vgood,1
1725,low,low,5more,more,big,low,unacc,0
1726,low,low,5more,more,big,med,good,0


## Feature hash

Feature hash es un enfoque para convertir una columna categórica en múltiples columnas usando trucos de hashing. Puede definir el número de nuevas columnas a las que convierte, que puede ser menor que el número de niveles en las columnas categóricas. En lugar de asignar 0 o 1 como una codificación de un solo dígito, el hashing de características utiliza más de dos valores (-1, 0 o 1 en el caso que se indica a continuación).

![alt text](3.png "Title")

Este etiquetado puede cubrir la deficiencia de la codificación de un solo paso que genera demasiadas columnas después de la transformación. Sin embargo, tener demasiadas columnas ya no es una cuestión fatal en las recientes técnicas avanzadas de modelado, por lo que el "hashing" de características no se utiliza ampliamente. Además, tener más de dos valores posibles en una nueva columna puede no ser bueno para algunos modelos.


Se puede utilizar la función FeatureHasher de scikit-learn para realizar este método.

In [5]:
from sklearn.feature_extraction import FeatureHasher

fh = FeatureHasher(n_features=3, input_type='string')
hashed = fh.transform(df[['safety']].astype(str).values)
hashed = pd.DataFrame(hashed.todense())
hashed

,0,1,2
0,0.0,0.0,1.0
1,-1.0,0.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,-1.0,0.0,0.0
...,...,...,...
1723,-1.0,0.0,0.0
1724,1.0,0.0,0.0
1725,0.0,0.0,1.0
1726,-1.0,0.0,0.0


In [6]:
hashed.columns = ['safety_FeaturedHasher' + str(i) for i in hashed.columns]
pd.concat([df, hashed], axis=1)

,buying,maint,doors,persons,lug_boot,safety,class value,lug_boot_Label_Encoder,safety_FeaturedHasher0,safety_FeaturedHasher1,safety_FeaturedHasher2
0,vhigh,vhigh,2,2,small,low,unacc,2,0.0,0.0,1.0
1,vhigh,vhigh,2,2,small,med,unacc,2,-1.0,0.0,0.0
2,vhigh,vhigh,2,2,small,high,unacc,2,1.0,0.0,0.0
3,vhigh,vhigh,2,2,med,low,unacc,1,0.0,0.0,1.0
4,vhigh,vhigh,2,2,med,med,unacc,1,-1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good,1,-1.0,0.0,0.0
1724,low,low,5more,more,med,high,vgood,1,1.0,0.0,0.0
1725,low,low,5more,more,big,low,unacc,0,0.0,0.0,1.0
1726,low,low,5more,more,big,med,good,0,-1.0,0.0,0.0



## Codificación binaria

La codificación binaria es un enfoque para convertir una columna categórica en múltiples columnas binarias, minimizando al mismo tiempo el número de nuevas columnas. En primer lugar, convertir el valor categórico en números enteros en algunos órdenes (por ejemplo, en orden alfabético o en orden de aparición para la fila superior). A continuación, se convierte en un dígito binario de tal manera que 1 a 1, 2 a 10, 5 a 101, etc. Finalmente, se diviode el dígito binario en columnas separadas, cada una de las cuales tiene un solo dígito (1 o 0).

![alt text](4.png "Title")

In [7]:
from category_encoders import BinaryEncoder

be = BinaryEncoder()
df_be = be.fit_transform(df['class value'])
pd.concat([df,df_be],axis=1)

,buying,maint,doors,persons,lug_boot,safety,class value,lug_boot_Label_Encoder,class value_0,class value_1,class value_2
0,vhigh,vhigh,2,2,small,low,unacc,2,0,0,1
1,vhigh,vhigh,2,2,small,med,unacc,2,0,0,1
2,vhigh,vhigh,2,2,small,high,unacc,2,0,0,1
3,vhigh,vhigh,2,2,med,low,unacc,1,0,0,1
4,vhigh,vhigh,2,2,med,med,unacc,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good,1,1,0,0
1724,low,low,5more,more,med,high,vgood,1,0,1,1
1725,low,low,5more,more,big,low,unacc,0,0,0,1
1726,low,low,5more,more,big,med,good,0,1,0,0


## Codificación Base N

La codificación de Base N es la generalización de la codificación binaria en el sentido de que, en lugar de utilizar la base 2, BaseN utiliza un número arbitrario como base. A medida que N aumenta el número de nuevas columnas se reduce, pero también significa que hay más información superpuesta dentro de las nuevas columnas que la que vimos en el caso de la codificación binaria, lo que potencialmente empeora el modelo final. Si N es infinito, la codificación de la BaseN es exactamente la misma que la de la etiqueta. Como ya se ha dicho, la codificación de etiquetas es inapropiada para la mayoría de los modelos.

![alt text](5.png "Title")


La función BaseNEncoder de la librería category_encoder puede ser útil para este método.

In [8]:
from category_encoders import BaseNEncoder

bne = BaseNEncoder(base=3)
df_bne = bne.fit_transform(df['class value'])
pd.concat([df, df_bne],axis=1)

,buying,maint,doors,persons,lug_boot,safety,class value,lug_boot_Label_Encoder,class value_0,class value_1,class value_2
0,vhigh,vhigh,2,2,small,low,unacc,2,0,0,1
1,vhigh,vhigh,2,2,small,med,unacc,2,0,0,1
2,vhigh,vhigh,2,2,small,high,unacc,2,0,0,1
3,vhigh,vhigh,2,2,med,low,unacc,1,0,0,1
4,vhigh,vhigh,2,2,med,med,unacc,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good,1,0,1,1
1724,low,low,5more,more,med,high,vgood,1,0,1,0
1725,low,low,5more,more,big,low,unacc,0,0,0,1
1726,low,low,5more,more,big,med,good,0,0,1,1


## Codificación de frecuencias

La codificación de frecuencias es un enfoque para transformar la columna categórica en una nueva columna con números enteros que representan las frecuencias de los niveles de la columna original. Esto puede funcionar bien cuando la frecuencia de los niveles es influyente para la variable objetivo.

![alt text](6.png "Title")

También se pueden convertir las frecuencias en rangos, al igual que la serialización de la codificación de frecuencias y la transformación de rangos. Tengan cuidado de que las frecuencias de los niveles tienen más probabilidades de causar empates que la clasificación de los datos originales.

No hay ninguna biblioteca (aún) que soporte esta codificación en python pero puede ser fácilmente implementada por la función nativa value_counts() de pandas.

In [9]:
freq = df['class value'].value_counts()
df['class value_fe'] = df['class value'].map(freq)
df
# test data should be applied the same encoding rule as training data.

,buying,maint,doors,persons,lug_boot,safety,class value,lug_boot_Label_Encoder,class value_fe
0,vhigh,vhigh,2,2,small,low,unacc,2,1210
1,vhigh,vhigh,2,2,small,med,unacc,2,1210
2,vhigh,vhigh,2,2,small,high,unacc,2,1210
3,vhigh,vhigh,2,2,med,low,unacc,1,1210
4,vhigh,vhigh,2,2,med,med,unacc,1,1210
...,...,...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good,1,69
1724,low,low,5more,more,med,high,vgood,1,65
1725,low,low,5more,more,big,low,unacc,0,1210
1726,low,low,5more,more,big,med,good,0,69
